In [ ]:
import glob, os
import pandas as pd
from Bio import SeqIO, SearchIO
import subprocess as sp

In [ ]:
def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

In [ ]:
genome_dir = "PATH_TO_GENOME_FASTAS"
out_dir = "PATH_TO_OUT_DIRECTORY"
reference_path = "PATH_TO_REFERENCES"
hmm_path = "PATH_TO_HMM_DIRECTORY"

### predict proteins

In [ ]:
# predict proteins prodigal single
# change genetic code to 25 for BD1-5 (Gracilibacteria)

if not os.path.isdir(out_dir + "prodigal/"):
    os.mkdir(out_dir + "prodigal/")
    
for genome in glob.glob(genome_dir + "/*"):
    
    call = "prodigal -i " + genome + " -m -g 11" + " -a " + \
        out_dir + "prodigal/" + os.path.basename(genome).replace(".fa",".faa")
    sp.call(call, shell=True)

In [ ]:
# create concatenated protein file for hmm
call = "cat " + out_dir + "prodigal/* > " + out_dir + "all_proteins.faa"
sp.call(call, shell=True)

### run hmms

In [ ]:
def parse_hmm(result_table):
    
    temp = {}
    count = 0
    
    # parse each result file using searchio
    for result in SearchIO.parse(result_table, "hmmer3-tab"):
        for item in result.hits:
            temp[count] = {"gene": item.id, "score": item.bitscore, "eval": item.evalue}
            count += 1
            
    return(pd.DataFrame.from_dict(temp, orient="index"))

In [ ]:
def run_hmm(hmm_name, threshold, protein_file):
    
    basename = os.path.basename(hmm_name).replace(".hmm", "")
    result_file = out_dir + "/hmm_results/" + basename + ".results"
    call = "hmmsearch -E " + threshold + " --cpu 16 --tblout " + result_file + " " + hmm_name + " " + protein_file
    sp.call(call, shell=True)

In [ ]:
total = 0

if not os.path.isdir(out_dir + "hmm_results/"):
    os.mkdir(out_dir + "hmm_results/")
    
for hmm in glob.glob(hmm_path + "*"):
        
    run_hmm(hmm, "1e-6", out_dir + "all_proteins.faa")
    total += 1

    print('hmms run: [%d]\r'%total, end="")

### process hmms

In [ ]:
## READ IN RESULTS

hmm_results = {}

for hmm_result in glob.glob(out_dir + "hmm_results/*.results"):
    hmm_results[os.path.basename(hmm_result).split(".")[0]] = parse_hmm(hmm_result) 

for hmm in hmm_results.keys():
    
    table = hmm_results[hmm]
    table["hmm"] = hmm

In [ ]:
#concatenate all hmms
all_results = pd.concat(list(hmm_results.values()))
# then recast as long
all_results_long = all_results.pivot("gene", "hmm", "score").fillna(0)
# select best hit per orf
all_results_long["best_hmm"] = all_results_long.idxmax(axis=1)
all_results_long["best_score"] = all_results_long.max(axis=1)
all_results_sub = all_results_long.reset_index()[["gene", "best_hmm", "best_score"]]

In [ ]:
# create scaf2bin
orf2bin = {}

for proteome in glob.glob(out_dir + "prodigal/*"):
    for orf in SeqIO.parse(open(proteome), "fasta"):
        orf2bin[orf.description.split(" ")[0]] = os.path.basename(proteome).split(".")[0]

In [ ]:
all_results_sub["bin"] = all_results_sub["gene"].map(orf2bin)
hmms_df = all_results_sub[["bin", "gene", "best_hmm", "best_score"]]
# select best hit for each hmm within a bin
hmms_dfp = pd.pivot_table(hmms_df,index=["bin","best_hmm"],columns="gene", values="best_score").fillna(0)
hmms_dfp["best_gene"] = hmms_dfp.idxmax(axis=1)
hmms_dfp = hmms_dfp.reset_index()[["bin", "best_hmm", "best_gene"]]

In [ ]:
# pivot one more time
hmms_final = hmms_dfp.pivot("bin", "best_hmm", "best_gene").fillna("None")
hmms_final = hmms_final.reset_index()

### extract and align sequences

In [ ]:
#write out sequences

if not os.path.isdir(out_dir + "markers/"):
    os.mkdir(out_dir + "markers/")

for hmm in hmm_results.keys():
    
    # get sequence names
    with open(out_dir + "markers/" + hmm + ".names.txt", "w") as outfile1:
        for gene in hmms_final[hmm].to_list():
            if gene != "None":
                outfile1.write(gene + "\n")
    
    # pull sequences
    call = "pullseq -n " + out_dir + "markers/" + hmm + ".names.txt -i " + \
        out_dir + "all_proteins.faa > " + out_dir + "markers/" + hmm + ".faa"
    sp.call(call, shell=True)

In [ ]:
with open(out_dir + "align_trim.sh","w") as outfile:

    for aln in glob.glob(out_dir + "markers/*faa"):
        
        name = os.path.basename(aln).split(".")[0]
        # find corresponding ref set
        ref_set = glob.glob(reference_path + "/" + name + "*")[0]
        # concatenate
        concat_file = out_dir + "markers/" + name + ".concat.faa"
        outfile.write("cat " + aln + " " + ref_set + " > " + concat_file + "\n")
        # mafft aln
        outfile.write("mafft --thread 16 --reorder " + concat_file + " > " + concat_file.replace(".faa", ".mafft") + "\n")
        ## bmge trim
        outfile.write("java -jar BMGE.jar -i " + concat_file.replace(".faa", ".mafft") + " -t AA -m BLOSUM30 -of " + concat_file.replace(".faa", ".trimmed.mafft") + "\n")

You then need to chmod +x the align_trim.sh file and run it (in screen for large dataset).

### read in and create concat alignment

In [ ]:
seq_db = {}
aln_lens = {}

#iterate through alignments
for aln in glob.glob(out_dir + "markers/*trimmed.mafft"):

    # get name of hmm
    hmm = os.path.basename(aln).split(".")[0]
    #iterate through sequences in each alignment
    for record in SeqIO.parse(open(aln), "fasta"):
        try:
            bin = orf2bin[record.description.split(" ")[0]]
        except:
            bin = record.description.split(" ")[0]
        #check if bin already represented in seq db
        if bin in seq_db:
            #if so, add new sequence
            seq_db[bin][hmm] = str(record.seq)
        # if not, create slot for this bin's seqs
        else:
            seq_db[bin] = {hmm: str(record.seq)}
        
        # finally, get aln length to use later
        aln_lens[hmm]= len(record.seq)

In [ ]:
# now we have the sequence for each marker stored for each bin
# lets turn it into a dataframe
seq_df = pd.DataFrame.from_dict(seq_db, orient="index").fillna("None")

In [ ]:
#finally, create the concatenated alignment
# open our blank alignment file
with open(out_dir + "rp16b_markers_concat.mafft", "w") as outfile:
    
    # now iterate through each row (bin) in dataframe and write out each marker
    for key, row in seq_df.iterrows():
        
        # first count the # of markers
        count = 0
        for column in seq_df.columns:
            if row[column] != "None":
                count+=1
        
        # min # of markers to include
        if count >= 7:
            
            # first write the bin name
            outfile.write(">" + key + "\n")
            # then iterate through each marker
            for marker in seq_df.columns.to_list():
                # check if gene present in matrix
                if row[marker] != "None":
                    #if so, write it out
                    outfile.write(row[marker])
                # if missing gene, put gaps corresponding
                # to aln length of that protein
                else:
                    outfile.write("-"*aln_lens[marker])
            # at the end, start a new line
            outfile.write("\n")        

### run the tree

In [ ]:
call = "iqtree -s " + out_dir + "rp16b_markers_concat.mafft" + \
    " -m TEST -st AA -bb 1500 -nt AUTO -ntmax 20 -pre " + out_dir + "rp16b_markers_concat"
print(call)